## VacationPy
_________________________________________________________________________________________________________________


### Starter Code to Import Libraries and Load the Weather and Coordinates Data


In [1]:
# Dependencies and Setup

import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,georgetown,5.4112,100.3354,78.73,93,20,0.00,MY,1682278317
1,1,cole harbour,44.6724,-63.4751,60.19,75,83,9.22,CA,1682278596
2,2,mtsensk,53.2766,36.5733,51.46,59,46,6.67,RU,1682278597
3,3,isafjordur,66.0755,-23.1240,36.00,60,75,5.01,IS,1682278597
4,4,grytviken,-54.2811,-36.5092,38.75,97,100,11.72,GS,1682278597


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
Humidity_plot_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 500,
    frame_height = 350, size = "Humidity",
    scale = 0.5,
    color = "City")

# Display the map
Humidity_plot_map

AttributeError: 'DataFrame' object has no attribute 'hvplot'

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data = city_data_df.loc[(city_data_df["Max Temp"]<=75) & (city_data_df["Max Temp"]>50)]
clean_city_data = city_data_df.loc[city_data_df["Wind Speed"]<3.5]
clean_city_data = city_data_df.loc[city_data_df["Cloudiness"] ==1]

# Drop any rows with null values
clean_city_data = clean_city_data.dropna()

# Display sample data
clean_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,minas de marcona,-15.2119,-75.1103,72.86,55,1,14.63,PE,1682278605
108,108,pisco,-13.7000,-76.2167,77.05,69,1,6.91,PE,1682278476
360,360,victorica,-36.2151,-65.4359,75.29,28,1,11.48,AR,1682278715
406,406,smara,26.7384,-11.6719,70.99,56,1,17.65,EH,1682278730
581,581,cova figueira,14.8905,-24.2934,69.64,73,1,18.77,CV,1682278789


### Step 3: Create a new DataFrame called hotel_df.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(clean_city_data, columns = ["City", "Country", "Lat", "Lng", "humidity"])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,humidity,Hotel Name
27,minas de marcona,PE,-15.2119,-75.1103,NaN,
108,pisco,PE,-13.7000,-76.2167,NaN,
360,victorica,AR,-36.2151,-65.4359,NaN,
406,smara,EH,26.7384,-11.6719,NaN,
581,cova figueira,CV,14.8905,-24.2934,NaN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key, "categories": "accommodation.hotel", "limit": 25}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
minas de marcona - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
victorica - nearest hotel: Hotel Mazamet
smara - nearest hotel: Hôtel Amine فندق الأمين
cova figueira - nearest hotel: No hotel found


,City,Country,Lat,Lng,humidity,Hotel Name
27,minas de marcona,PE,-15.2119,-75.1103,NaN,No hotel found
108,pisco,PE,-13.7000,-76.2167,NaN,La Portada
360,victorica,AR,-36.2151,-65.4359,NaN,Hotel Mazamet
406,smara,EH,26.7384,-11.6719,NaN,Hôtel Amine فندق الأمين
581,cova figueira,CV,14.8905,-24.2934,NaN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plot_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 500,
frame_height = 350, size = "Humidity",
scale = 0.5,
color = "City" hover_cols = ["Hotel Name", "country"], size "humidity")

# Display the map
plot_map

SyntaxError: invalid syntax. Perhaps you forgot a comma? (185393525.py, line 5)